In [4]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from PIL import Image, ImageTk
import os

class BitmapFontVisualizer:
    def __init__(self, root):
        self.root = root
        self.root.title("Bitmap Font Visualizer")
        
        # Data storage
        self.font_data = None
        self.page_images = {}
        self.current_char_index = 0
        self.chars = []
        self.zoom_level = 1.0
        self.fnt_path = None
        self.bg_color = 'white'  # Default background color
        self.modified = False  # Track if changes have been made
        self.padding = 1  # 1 pixel padding

        self.setup_ui()
        
        # Bind mouse wheel for zooming
        self.canvas.bind('<MouseWheel>', self.on_mousewheel)  # Windows
        self.canvas.bind('<Button-4>', self.on_mousewheel)    # Linux scroll up
        self.canvas.bind('<Button-5>', self.on_mousewheel)    # Linux scroll down
    
    def setup_ui(self):
        # Main container
        main_container = ttk.PanedWindow(self.root, orient=tk.HORIZONTAL)
        main_container.grid(row=0, column=0, sticky="nsew")
        
        # Left panel for visualization
        left_panel = ttk.Frame(main_container)
        main_container.add(left_panel)
        
        # Right panel for editing
        right_panel = ttk.Frame(main_container)
        main_container.add(right_panel)
        
        # Top frame for controls in left panel
        control_frame = ttk.Frame(left_panel, padding="5")
        control_frame.grid(row=0, column=0, sticky="ew")
        
        # Load and save buttons
        ttk.Button(control_frame, text="Load Font File", command=self.load_font).grid(row=0, column=0, padx=5)
        ttk.Button(control_frame, text="Save Font File", command=self.save_font).grid(row=0, column=1, padx=5)
        
        # Navigation buttons
        ttk.Button(control_frame, text="Previous", command=self.prev_char).grid(row=0, column=2, padx=5)
        ttk.Button(control_frame, text="Next", command=self.next_char).grid(row=0, column=3, padx=5)
        
        # Zoom controls
        zoom_frame = ttk.Frame(control_frame)
        zoom_frame.grid(row=0, column=4, padx=5)
        ttk.Label(zoom_frame, text="Zoom:").pack(side=tk.LEFT)
        ttk.Button(zoom_frame, text="-", width=2, command=lambda: self.set_zoom(self.zoom_level - 0.5)).pack(side=tk.LEFT)
        ttk.Button(zoom_frame, text="+", width=2, command=lambda: self.set_zoom(self.zoom_level + 0.5)).pack(side=tk.LEFT)
        
        
        self.bg_toggle = ttk.Button(control_frame, text="Toggle Background", command=self.toggle_background)
        self.bg_toggle.grid(row=0, column=5, padx=5)

        # Character info frame
        self.info_frame = ttk.LabelFrame(left_panel, text="Character Info", padding="5")
        self.info_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=5)
        
        # Labels for character information
        self.char_id_label = ttk.Label(self.info_frame, text="ID: ")
        self.char_id_label.grid(row=0, column=0, sticky="w")
        
        # Canvas for displaying the character
        self.canvas = tk.Canvas(left_panel, width=400, height=400, bg='white')
        self.canvas.grid(row=2, column=0, sticky="nsew", padx=5, pady=5)
        
        # Editor frame in right panel
        editor_frame = ttk.LabelFrame(right_panel, text="Character Editor", padding="5")
        editor_frame.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        
        # Create entry fields for editing
        self.editor_vars = {}
        fields = ['x', 'y', 'width', 'height', 'xoffset', 'yoffset', 'xadvance']
        def validate_number(value):
            if value == "": return True
            try:
                int(value)
                return True
            except ValueError:
                return False
        
        validation = self.root.register(validate_number)
        
        for i, field in enumerate(fields):
            ttk.Label(editor_frame, text=field).grid(row=i, column=0, sticky="w", padx=5)
            var = tk.StringVar()
            self.editor_vars[field] = var
            entry = ttk.Entry(editor_frame, textvariable=var, width=10, 
                            validate='key', validatecommand=(validation, '%P'))
            entry.grid(row=i, column=1, sticky="w", padx=5)
            entry.bind('<Return>', lambda e, f=field: self.update_char_field(f))
            entry.bind('<FocusOut>', lambda e, f=field: self.update_char_field(f))

        # Apply button
        ttk.Button(editor_frame, text="Apply Changes", command=self.apply_changes).grid(row=len(fields), column=0, columnspan=2, pady=10)
        
        # Configure grid weights
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)
        left_panel.grid_rowconfigure(2, weight=1)
        left_panel.grid_columnconfigure(0, weight=1)
        
    def toggle_background(self):
        self.bg_color = 'black' if self.bg_color == 'white' else 'white'
        self.canvas.configure(bg=self.bg_color)
        self.display_current_char()
        
    def set_zoom(self, new_zoom):
        self.zoom_level = max(0.5, min(10.0, new_zoom))
        self.display_current_char()

    def on_mousewheel(self, event):
        if event.num == 5 or event.delta < 0:  # Scroll down
            self.set_zoom(self.zoom_level - 0.1)
        else:  # Scroll up
            self.set_zoom(self.zoom_level + 0.1)

    def update_char_field(self, field):
        if not self.chars:
            return
        try:
            value = int(self.editor_vars[field].get())
            if value != self.chars[self.current_char_index][field]:
                self.chars[self.current_char_index][field] = value
                self.modified = True
                self.display_current_char()
        except ValueError:
            # If invalid value, restore the original value
            self.editor_vars[field].set(str(self.chars[self.current_char_index][field]))
       
    def apply_changes(self):
        if not self.chars:
            return
        try:
            for field in self.editor_vars:
                value = int(self.editor_vars[field].get())
                if value != self.chars[self.current_char_index][field]:
                    self.chars[self.current_char_index][field] = value
                    self.modified = True
            self.display_current_char()
        except ValueError as e:
            messagebox.showerror("Error", f"Invalid value in one of the fields: {str(e)}")

    def parse_fnt_file(self, filepath):
        pages = {}
        chars = []
        header_lines = []
        
        with open(filepath, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            parts = line.split(' ')
            line_type = parts[0]
            
            if line_type in ['info', 'common']:
                header_lines.append(line)
            elif line_type == 'page':
                header_lines.append(line)
                data = self.parse_key_value_pairs(parts[1:])
                pages[int(data['id'])] = data['file'].strip('"')
            elif line_type == 'chars':
                header_lines.append(line)
            elif line_type == 'char':
                data = self.parse_key_value_pairs(parts[1:])
                if data:
                    chars.append({
                        'id': int(data.get('id', 0)),
                        'x': int(data.get('x', 0)),
                        'y': int(data.get('y', 0)),
                        'width': int(data.get('width', 0)),
                        'height': int(data.get('height', 0)),
                        'xoffset': int(data.get('xoffset', 0)),
                        'yoffset': int(data.get('yoffset', 0)),
                        'xadvance': int(data.get('xadvance', 0)),
                        'page': int(data.get('page', 0))
                    })
        
        return pages, chars, header_lines

    def save_font(self):
        if not self.fnt_path or not self.chars:
            messagebox.showerror("Error", "No font file loaded")
            return
            
        if not self.modified:
            messagebox.showinfo("Info", "No changes to save")
            return
            
        try:
            with open(self.fnt_path, 'w', encoding='utf-8') as file:
                # Write header lines
                for line in self.header_lines:
                    file.write(line + '\n')
                
                # Write character data
                for char in self.chars:
                    line = f"char id={char['id']} x={char['x']} y={char['y']} "
                    line += f"width={char['width']} height={char['height']} "
                    line += f"xoffset={char['xoffset']} yoffset={char['yoffset']} "
                    line += f"xadvance={char['xadvance']} page={char['page']} chnl=15\n"
                    file.write(line)
                    
            self.modified = False
            messagebox.showinfo("Success", "Font file saved successfully")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save font file: {str(e)}")

    def parse_key_value_pairs(self, parts):
        data = {}
        for part in parts:
            if '=' in part:
                key, value = part.split('=')
                data[key] = value
        return data
    
    def load_font(self):
        fnt_path = filedialog.askopenfilename(filetypes=[("Font files", "*.fnt")])
        if not fnt_path:
            return
            
        try:
            # Parse the .fnt file
            pages, chars, header_lines = self.parse_fnt_file(fnt_path)
            self.fnt_path = fnt_path
            self.header_lines = header_lines
            
            # Load all page images
            font_dir = os.path.dirname(fnt_path)
            for page_id, file_name in pages.items():
                img_path = os.path.join(font_dir, file_name)
                try:
                    self.page_images[page_id] = Image.open(img_path)
                except FileNotFoundError:
                    messagebox.showerror("Error", f"Could not find texture file: {file_name}")
                    return
            
            self.chars = chars
            self.current_char_index = 0
            self.display_current_char()
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load font: {str(e)}")

    def display_current_char(self):
        if not self.chars:
            return
            
        char = self.chars[self.current_char_index]
        
        # Update info labels and editor fields
        self.char_id_label.config(text=f"ID: {char['id']} (ASCII: {chr(char['id']) if 32 <= char['id'] <= 126 else 'N/A'})")
        
        for field in self.editor_vars:
            self.editor_vars[field].set(str(char[field]))
        
        # Clear canvas
        self.canvas.delete("all")
        
        # Get character image WITH padding where available
        page_img = self.page_images[char['page']]
        
        # Calculate actual available padding on each side
        actual_left_padding = min(self.padding, char['x'])
        actual_right_padding = min(self.padding, page_img.width - (char['x'] + char['width']))
        actual_top_padding = min(self.padding, char['y'])
        actual_bottom_padding = min(self.padding, page_img.height - (char['y'] + char['height']))
        
        # Crop the character with available padding
        char_img = page_img.crop((
            char['x'] - actual_left_padding,
            char['y'] - actual_top_padding,
            char['x'] + char['width'] + actual_right_padding,
            char['y'] + char['height'] + actual_bottom_padding
        ))
        
        if char_img.mode != 'RGBA':
            char_img = char_img.convert('RGBA')
        
        # Create a new image with full padding
        padded_width = char['width'] + 2 * self.padding
        padded_height = char['height'] + 2 * self.padding
        padded_img = Image.new('RGBA', (padded_width, padded_height), (64, 64, 64, 255))  # Dark gray color
        
        # Calculate where to paste the character image with actual padding
        paste_x = self.padding - actual_left_padding
        paste_y = self.padding - actual_top_padding
        
        # Paste the character with its actual padding onto the fully padded image
        padded_img.paste(char_img, (paste_x, paste_y))
        
        # Apply zoom
        if self.zoom_level != 1.0:
            new_size = (
                int(padded_width * self.zoom_level),
                int(padded_height * self.zoom_level)
            )
            padded_img = padded_img.resize(new_size, Image.NEAREST)
        
        self.current_photo = ImageTk.PhotoImage(padded_img)
        
        # Calculate canvas center
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        center_x = canvas_width // 2
        center_y = canvas_height // 2
        
        # Calculate zoomed dimensions
        zoomed_width = char['width'] * self.zoom_level
        zoomed_height = char['height'] * self.zoom_level
        zoomed_padding = self.padding * self.zoom_level
        
        # Calculate positions for the character bounds
        box_x = center_x - zoomed_width // 2
        box_y = center_y - zoomed_height // 2
        
        # Calculate position for padded image
        image_x = box_x - zoomed_padding
        image_y = box_y - zoomed_padding
        
        # Draw baseline
        baseline_color = 'yellow' if self.bg_color == 'black' else 'blue'
        self.canvas.create_line(
            center_x - 100, center_y, 
            center_x + 100, center_y, 
            fill=baseline_color, dash=(4, 4)
        )
        
        # Draw character image (including padding)
        self.canvas.create_image(
            image_x, image_y,
            image=self.current_photo,
            anchor='nw'
        )
        
        # Draw actual character bounds
        border_color = 'cyan' if self.bg_color == 'black' else 'red'
        self.canvas.create_rectangle(
            box_x, box_y,
            box_x + zoomed_width,
            box_y + zoomed_height,
            outline=border_color
        )
        
        # Draw padding boundary
        padding_color = 'gray50'
        self.canvas.create_rectangle(
            box_x - zoomed_padding,
            box_y - zoomed_padding,
            box_x + zoomed_width + zoomed_padding,
            box_y + zoomed_height + zoomed_padding,
            outline=padding_color,
            dash=(2, 2)
        )
        
        # Draw offset indicators
        offset_color = 'lime' if self.bg_color == 'black' else 'green'
        self.canvas.create_line(
            center_x, center_y,
            center_x + char['xoffset'] * self.zoom_level,
            center_y + char['yoffset'] * self.zoom_level,
            fill=offset_color, arrow=tk.LAST
        )
        
    def next_char(self):
        if self.chars and self.current_char_index < len(self.chars) - 1:
            self.current_char_index += 1
            self.display_current_char()
    
    def prev_char(self):
        if self.chars and self.current_char_index > 0:
            self.current_char_index -= 1
            self.display_current_char()

if __name__ == "__main__":
    root = tk.Tk()
    app = BitmapFontVisualizer(root)
    root.mainloop()